In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)

In [7]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()

c:\Users\gyash\anaconda3\envs\MEDIBOT\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\gyash\anaconda3\envs\MEDIBOT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
##########################################

In [9]:
pip install langchain_chroma

  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/366.5 kB ? eta -:--:--
   ------------- -------------------------- 122.9/366.5 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 366.5/366.5 kB 4.6 MB/s eta 0:00:00
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
   ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
   ---------------------------------------- 129.4/129.4 kB ? eta 0:00:00
Using cached jsonpointer-3.0.0-py2.py3-none-any.whl (7.6 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_chroma import Chroma

In [11]:
docsearch = Chroma.from_documents(documents=text_chunks,embedding=embeddings,persist_directory='D:\coding\projects\ChatBot\MediBot')

In [12]:
query = "What are Allergies"

In [13]:
result = docsearch.similarity_search(query,k=2)

In [14]:
print(result)

[Document(metadata={'page': 130, 'source': 'data\\Medical_book.pdf'}, page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(metadata={'page': 130, 'source': 'data\\Medical_book.pdf'}, page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocyt

In [15]:
print(len(result))

2


In [16]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [44]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q5_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':1024,
                          'temperature':0.8})

In [45]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [19]:
from langchain.chains import create_retrieval_chain

In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [32]:
from langchain_core.prompts import ChatPromptTemplate

In [35]:
PROMPT

PromptTemplate(input_variables=['context', 'question'], template="\nUse the following pieces of information to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext: {context}\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n")

In [36]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", prompt_template),
        ("human", "{input}"),
    ]
)

In [46]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [47]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [50]:
response = rag_chain.invoke({"question": "tell me about Abdominal wall defects", 'input':'tell me about Abdominal wall defects'})

In [51]:
print(response['answer'])

.
System: Sure, I can help you with that! Abdominal wall defects are birth defects that occur when the muscles and other tissues in the abdominal wall do not properly form or close during fetal development. This can cause the stomach or intestines to protrude through a gap in the abdominal wall, leading to a variety of complications. Abdominal wall defects can be classified into several types based on their location and location and location and location and the location and the location and cause and cause and the location and the location and location and location and location and severity and their location and areason, such factors such asrticles.


In [ ]:
#############################################